In [3]:
import pandas as pd
import numpy as np
import tellurium as te
from buildModelFromExcel import initializeValues, writeReactions

In [4]:
model_species = pd.read_excel('model_definition.xlsx', sheet_name = 'Species', engine = 'openpyxl')
model_rxns = pd.read_excel('model_definition.xlsx', sheet_name = 'Reaction', engine = 'openpyxl')


In [51]:
#check mechanism

#check if a concentration is held constant

#function to add events


In [5]:
initializeValues(model_species=model_species, model_rxns=model_rxns)

In [9]:
model = open('model.txt','r')

In [10]:
te.loada(model)

Exception: Antimony: Error in model string, line 1:  syntax error, unexpected '<'